Preguntas
* Distinguir entre materiales y texturas
* Obtener la url desde el data
* Software compatible ¿lista?
* Tags ¿lista?

In [2]:

import requests
from lxml import html
import csv 
import re

def getTextures(tree, xpathCode):
    chain = tree.xpath(xpathCode)
    txt_chain = chain[0].text_content()
    num_elementos = len(txt_chain.split(','))
    print(num_elementos)
    
def getImportant(tree, xpathCode):
    dicc = {}
    propierties = tree.xpath(xpathCode)
    for element in propierties:
        text = element.text_content()
        print(text)
        
        '''Encontrar el ID'''
        patronID = r'Model ID: #(\d+)'
        searchID = re.search(patronID, text)
        if searchID:
            dicc['ID'] = searchID.group(1)
        
        
        '''Encontrar el tamaño'''
        patronSize = r'File Size: ([\d.]+ \w+)'
        searchSize = re.search(patronSize, text)
        if searchSize:
            dicc['Size'] = searchSize.group(1)
        
    print(dicc)
'''with open("Modelos_3D.csv", mode="w", newline='') as archivo: 
            escritura = csv.writer(archivo)
            escritura.writerow(Datos)'''

Datos=['path', 
       'file_size', #
       'id', #
       'name', 
       'description',
       'uri',
       'aviable_formats',
       'textures', 
       'materials', 
       'polygons', 
       'vertex',
       'is_rigged', 
       'is_animated',
       'license', 
       'quality',
       'software',
       'category',
       'tags'
       ]

url = 'https://open3dmodel.com/3d-models/3d-character-human-body-3dsmax-model_20474.html'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

try:
    
    response = requests.get(url, timeout=6, headers=headers)
    tree = html.fromstring(response.content)

    
    textures = getTextures(tree, '/html/body/div[3]/article/div[3]/div[1]/div[3]/div[2]/div[5]/p[5]')
    propierties = getImportant(tree, '/html/body/div[3]/div/div[5]/ul/li')
    
        

except requests.RequestException as e:
    print(f"Error en la solicitud: {e}")


4
Model ID: #20474 
Formats: .3ds, .Max 
Textures: Yes 
Materials: Yes 
File Size: 7.7 MB 
Polygons: N/A 
Vertices: N/A 
Quality: Medium Poly 
Rigged: No 
Animated: No 
License: Non-commercial
{'ID': '20474', 'Size': '7.7 MB'}
